In [ ]:
#!pip install azure-identity azure-mgmt-cosmosdb

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.cosmosdb import CosmosDBManagementClient
from azure.mgmt.cosmosdb.models import DatabaseAccountCreateUpdateParameters, Location

In [ ]:
subscription_id = "23d99a1f-8507-41a7-b40b-64402a51e95e"
resource_group = 'pvn-rg'
account_name = "pvn-cosmos"
location = "West US 2"

In [ ]:
credential = DefaultAzureCredential()
cosmos_client = CosmosDBManagementClient(credential, subscription_id)

In [ ]:
params = DatabaseAccountCreateUpdateParameters(location=location,
                                               locations=[Location(location_name=location, failover_priority=0)],
                                               kind='GlobalDocumentDB',
                                               create_mode='Default',
                                               enable_free_tier=True)

In [ ]:
cosmos_client.database_accounts.begin_create_or_update(resource_group, account_name, params).result()
print("Successfully created db account {} in resource group {}".format(account_name, resource_group))

In [ ]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions
import uuid

# Configuration
ENDPOINT = "https://pvn-cosmos.documents.azure.com:443/"
KEY = "MumQzNM2JoaBs0IrQp4UQKeEZvS5Nq3tx0wsRLy44dKOOuSkoKWMeKcd8lA6oMeEYMPHO3jPu36oACDbqzYnyw=="
#DATABASE_NAME = "Blobdb"
#CONTAINER_NAME = "Users"

In [ ]:
client = CosmosClient(ENDPOINT, KEY)

In [ ]:
'''database_name = "blogdb"
database = client.create_database_if_not_exists(id=database_name)'''

In [ ]:
DATABASE_NAME="Blogdb"

In [ ]:
database = client.get_database_client(DATABASE_NAME)

In [ ]:
container_name = "Users"
partition_key_path = "/userid"

In [ ]:
from azure.cosmos import PartitionKey

container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path=partition_key_path),
    offer_throughput=400)

In [ ]:
container_name = "Session"
partition_key_path = "/sessionid"

In [ ]:
container_name = "Products"
partition_key_path = "/productid"

In [ ]:
# List all containers
print("Containers in database:")
for container in database.list_containers():
    print("-", container['id'])

In [ ]:
container = database.get_container_client("Products")

In [ ]:
import uuid

# Sample product item
product_item = {
    "id": str(uuid.uuid4()),      # unique document id
    "productid": "P1001",         # partition key value
    "name": "Laptop",
    "price": 75000,
    "category": "Electronics"
}

container.create_item(body=product_item)
print("Inserted item:", product_item)

In [ ]:
# Query all items
print("\nReading items in container:")
for item in container.read_all_items():
    print(item)

# OR Query using SQL syntax
query = "SELECT * FROM c WHERE c.category='Electronics'"
items = list(container.query_items(query=query, enable_cross_partition_query=True))
print("\nQueried items:", items)

In [ ]:
# Let's pick an item by productid
query = "SELECT * FROM c WHERE c.productid='P1001'"
items = list(container.query_items(query=query, enable_cross_partition_query=True))
if items:
    item = items[0]
    print("Before update:", item)
    
    # Modify the item
    item["price"] = 78000
    item["stock"] = 25

    # Replace the item
    container.replace_item(item=item['id'], body=item)
    print("Updated item:", item)

In [ ]:
prod_container = database.get_container_client("Products")

In [ ]:
ses_container = database.get_container_client("Session")

In [ ]:
users_container = database.get_container_client("Users")

In [ ]:
users=list(users_container.query_items(
    query="SELECT * FROM c",
    enable_cross_partition_query=True))

In [ ]:
products=list(prod_container.query_items(
    query="SELECT * FROM c",
    enable_cross_partition_query=True))

In [ ]:
products

In [ ]:
joined_data = []
for user in users:
    user_products = [p for p in products if p['userid']==user["userid"]]
    joined_data.append({"user":user, "products":user_products})
print(joined_data)

In [ ]:
item_id = item["id"]
partition_key = item["productid"]

container.delete_item(item=item_id, partition_key=partition_key)
print(f"Deleted item with id: {item_id}")

In [ ]:
remaining_items = list(container.read_all_items())
print("\nRemaining items after deletion:")
for it in remaining_items:
    print(it)

In [ ]:
cosmos_client.database_accounts.begin_delete(resource_group, account_name).wait()